# Question 3, work in progress

In [ ]:
import numpy as np

# Assumption of no clouds

In [60]:
# SHORT WAVE

P_sol= 341.3
r_SM = 0.1065

a_O3 = 0.08

P_abs_ozone = a_O3*P_sol

P_after_ozone = P_sol - P_abs_ozone


a_SW = 0.1451

r_SE = 0.17

P_refl_atm = r_SM*P_after_ozone

P_abs_atm = (a_SW)*(1-r_SM)*P_after_ozone

P_trans_atm = (1 - (a_O3 + a_SW))*(1- r_SM)*P_after_ozone

P_refl_earth = r_SE * P_trans_atm
P_abs_earth = (1 - r_SE)*P_trans_atm

P_abs_atm = P_abs_atm + a_SW*(1-r_SE)*P_refl_earth 

P_refl_atm = P_refl_atm + r_SM*(1-a_SW)*(1-r_SE)*P_refl_earth 

P_abs_ozone = P_abs_ozone + a_O3*P_refl_atm

P_abs_earth = P_abs_earth + (1-r_SE)*(r_SM*(1-a_SW)*(1-r_SE)*P_refl_earth )



print(P_refl_atm)
print(P_abs_atm)
print(P_trans_atm)
print(P_refl_earth)
print(P_abs_earth)

print("TOTAL", P_refl_atm + P_abs_atm + P_refl_earth + P_abs_earth + P_abs_ozone , P_sol)

# LONG WAVE

sigma = 5.67*10**(-8) #W m-2 K-4



epsilon_A = 0.875

f_A = 0.618


alpha = 3
beta = 4

a_LW = 0.8258
#P_E0 = sigma*T_E**4
#P_A0 = epsilon_A*sigma*T_A**4
#P_A0_down = f_A*P_A0
#P_A0_up = (1-f_A)*P_A0
#P_EtoA = - alpha*(T_A -T_E)
#P_latent = -beta*(T_A-T_E)



from scipy.optimize import fsolve

def eqns(variables):
    T_E, T_A = variables

    P_non_rad = - (alpha + beta) * (T_A - T_E)
    #               P_abs_earth   +     P_A0_down    -    (P_EtoA + P_latent)    - P_E0
    eqn1 = P_abs_earth + f_A*epsilon_A*sigma* T_A**4 + P_non_rad - sigma* T_E**4

    #     P_abs_atm  +       a_LW*P_E0    +  (P_EtoA + P_latent)     - P_A0
    eqn2 = (P_abs_atm + P_abs_ozone) + a_LW*sigma* T_E**4 - P_non_rad - epsilon_A*sigma* T_A**4

    return [eqn1, eqn2]

T_init_guess = [298.0, 278.0]
solution, info , ier, mesg = fsolve(eqns, T_init_guess, xtol=1e-8, full_output=True)

print("Solution: ", solution)



36.23347969737066
45.15960425522606
217.40239960739996
36.958407933257995
182.7621034029596
TOTAL 331.316273664604 341.3
Solution:  [334.29583657 290.39195297]


# Assumption with clouds

In [36]:
CC = 0.66
r_SC = 0.22
r_LC = 0.195
a_LC = 0.622
a_SC = 0.1239

                # atm refect    + clouds reflect
P_refl_atm = r_SM*(1-CC)*P_sol + r_SC*CC*P_sol
print(P_refl_atm)
                #
P_abs_atm =  (a_O3 + a_SW)*(1-CC)*(1 -r_SM)*P_sol  + (a_O3 + a_SC)*(1-r_SC)*CC*P_sol
print(P_abs_atm)

P_trans_atm = P_sol - P_refl_atm - P_abs_atm
print(P_trans_atm)

P_refl_earth = r_SE * P_trans_atm
P_abs_earth = (1 - r_SE)*P_trans_atm

a_LW_total = a_LW*(1-CC) + a_LC*CC
epsilon_A_total = epsilon_A*(1-CC) + a_LC*CC

def eqns(variables):
    T_E, T_A = variables

    P_non_rad = - (alpha + beta) * (T_A - T_E)
    #               P_abs_earth   +     P_A0_down    -    (P_EtoA + P_latent)    - P_E0
    eqn1 = P_abs_earth + f_A*epsilon_A_total*sigma* T_A**4 + P_non_rad - sigma* T_E**4

    #     P_abs_atm  +       a_LW*P_E0    +  (P_EtoA + P_latent)     - P_A0
    eqn2 = P_abs_atm + a_LW_total*sigma* T_E**4 - P_non_rad - epsilon_A_total*sigma* T_A**4

    return [eqn1, eqn2]

T_init_guess = [298.0, 288.0]
solution, info , ier, mesg = fsolve(eqns, T_init_guess, xtol=1e-8, full_output=True)

print("Solution: ", solution)



61.915233
59.16464476370001
220.2201222363
Solution:  [300.61406609 283.4579788 ]
